In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('casting').getOrCreate()

24/06/03 21:03:02 WARN Utils: Your hostname, Ugurs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.57 instead (on interface en0)
24/06/03 21:03:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/03 21:03:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand

In [19]:
df_int_path = "/Users/ugurkalkavan/tmp/df_int"
df_int_sorted_path = "/Users/ugurkalkavan/tmp/df_int_sorted"
df_str_path = "/Users/ugurkalkavan/tmp/df_str"
df_str_sorted_path = "/Users/ugurkalkavan/tmp/df_str_sorted"

In [21]:
from pyspark.sql import SparkSession
import time

# Create a DataFrame with 10k rows of random numbers
df_int = spark.range(0, 100000000).select((rand()*10000000000).cast('int').alias('age'))



df_int.repartition(1).write.mode('overwrite').parquet(df_int_path)

df_int_sorted = df_int.sort("age")



df_int_sorted.repartition(1).write.mode('overwrite').parquet(df_int_sorted_path)

# Create a DataFrame with 10k rows of random numbers but cast to string
df_str = df_int.select(col('age').cast('string'))



df_str.repartition(1).write.mode('overwrite').parquet(df_str_path)

df_str_sorted = df_str.sort("age")

df_str_sorted.repartition(1).write.mode('overwrite').parquet(df_str_sorted_path)

In [35]:
spark.conf.set ("spark.sql.files.maxPartitionBytes", "134217728b") #10000b

In [39]:
import time

spark.sparkContext.setJobDescription('df_int')  # Setting Job description

# Read the integer DataFrame from Parquet file
start_time = time.time()

df_int = spark.read.parquet(df_int_path)

# Perform operation on integer DataFrame
df_int.filter(df_int.age < 10008).count()
print("Nr. of rdd partitions:", df_int.rdd.getNumPartitions())
# End timer and print time
end_time = time.time()

print(f"Time taken with integer from parquet: {round(end_time - start_time, 2)} seconds")

######
spark.sparkContext.setJobDescription('df_int_sorted')  # Setting Job description
# Read the integer DataFrame from Parquet file
start_time = time.time()

df_int_sorted = spark.read.parquet(df_int_sorted_path)

# Perform operation on integer DataFrame
df_int_sorted.filter(df_int_sorted.age < 10008).count()
print("Nr. of rdd partitions:", df_int_sorted.rdd.getNumPartitions())
# End timer and print time
end_time = time.time()

print(f"Time taken with integer sorted from parquet: {round(end_time - start_time, 2)} seconds")

#####
spark.sparkContext.setJobDescription('df_str')  # Setting Job description

# Read the string DataFrame from Parquet file
start_time = time.time()

df_str = spark.read.parquet(df_str_path)

# Perform operation on string DataFrame
df_str.filter(df_str.age < "10008").count()
print("Nr. of rdd partitions:", df_str.rdd.getNumPartitions())
# End timer and print time
end_time = time.time()

print(f"Time taken with string from parquet: {round(end_time - start_time, 2)} seconds")

######
spark.sparkContext.setJobDescription('df_str_sorted')  # Setting Job description

# Read the integer DataFrame from Parquet file
start_time = time.time()

df_str_sorted = spark.read.parquet(df_str_sorted_path)

# Perform operation on integer DataFrame
df_str_sorted.filter(df_str_sorted.age < "10008").count()
print("Nr. of rdd partitions:", df_str_sorted.rdd.getNumPartitions())

# End timer and print time
end_time = time.time()

print(f"Time taken with string sorted from parquet: {round(end_time - start_time, 2)} seconds")

Nr. of rdd partitions: 10
Time taken with integer from parquet: 0.9 seconds
Nr. of rdd partitions: 10
Time taken with integer sorted from parquet: 0.31 seconds


Nr. of rdd partitions: 10
Time taken with string from parquet: 3.19 seconds
Nr. of rdd partitions: 10
Time taken with string sorted from parquet: 0.51 seconds


In [25]:
spark

In [43]:
import os
import sys
import pyarrow as pa
import pyarrow.parquet as pq
print(f"{sys.version}")
print(f"{pa.__version__=}")
par_str_path = f"{df_str_path}/part-00000-5cd38d6b-3135-40e8-8faa-64389b5e5a18-c000.snappy.parquet"
par_int_path = f"{df_int_path}/part-00000-dd315e0b-beac-43f8-8f2b-1563c3bb7676-c000.snappy.parquet"
par_int_sorted_path = f"{df_int_sorted_path}/part-00000-853084f4-70f9-4923-b34d-f7c746d35cc0-c000.snappy.parquet"

3.11.7 (main, Dec 15 2023, 12:09:56) [Clang 14.0.6 ]
pa.__version__='14.0.2'


In [45]:
import pyarrow.parquet as pq

pq_file_int = pq.ParquetFile(par_int_path)
pq_file_str = pq.ParquetFile(par_str_path)
pq_file_int_sorted = pq.ParquetFile(par_int_sorted_path)
# Get metadata for the i-th RowGroup
rg_meta_int = pq_file_int.metadata.row_group(0);
rg_meta_str = pq_file_str.metadata.row_group(0);
rg_meta_int_sorted = pq_file_int_sorted.metadata.row_group(0);
# Get the "max" statistic for the k-th column
rg_meta_int = rg_meta_int.column(0)
rg_meta_str = rg_meta_str.column(0)
rg_meta_int_sorted = rg_meta_int_sorted.column(0)

In [47]:
def get_compression_ratio(uncompressed_size, compressed_size):
    ratio = uncompressed_size / compressed_size
    return ratio

In [56]:
print(rg_meta_str)
get_compression_ratio(rg_meta_str.total_uncompressed_size,rg_meta_str.total_compressed_size)

  file_offset: 719180
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 44291537
  path_in_schema: age
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 1000000266
      max: 999999896
      null_count: 0
      distinct_count: None
      num_values: 44291537
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('BIT_PACKED', 'PLAIN_DICTIONARY', 'RLE', 'PLAIN')
  has_dictionary_page: True
  dictionary_page_offset: 4
  data_page_offset: 719180
  total_compressed_size: 134821291
  total_uncompressed_size: 611882556


4.538471271573864

In [58]:
print(rg_meta_int)
get_compression_ratio(rg_meta_int.total_uncompressed_size,rg_meta_int.total_compressed_size)

  file_offset: 1046176
  file_path: 
  physical_type: INT32
  num_values: 84836597
  path_in_schema: age
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 285
      max: 2147483647
      null_count: 0
      distinct_count: None
      num_values: 84836597
      physical_type: INT32
      logical_type: None
      converted_type (legacy): NONE
  compression: SNAPPY
  encodings: ('BIT_PACKED', 'PLAIN_DICTIONARY', 'RLE', 'PLAIN')
  has_dictionary_page: True
  dictionary_page_offset: 4
  data_page_offset: 1046176
  total_compressed_size: 135239312
  total_uncompressed_size: 337905926


2.4985776768814087

In [60]:
print(rg_meta_int_sorted)
get_compression_ratio(rg_meta_int_sorted.total_uncompressed_size,rg_meta_int_sorted.total_compressed_size)

  file_offset: 4
  file_path: 
  physical_type: INT32
  num_values: 100000000
  path_in_schema: age
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 28
      max: 2147483647
      null_count: 0
      distinct_count: None
      num_values: 100000000
      physical_type: INT32
      logical_type: None
      converted_type (legacy): NONE
  compression: SNAPPY
  encodings: ('BIT_PACKED', 'RLE', 'PLAIN')
  has_dictionary_page: False
  dictionary_page_offset: None
  data_page_offset: 4
  total_compressed_size: 100874685
  total_uncompressed_size: 400180992


3.967110202128512

In [133]:
spark